In [ ]:
from IPython.display import Image
from IPython.core.display import HTML

Image(url= "http://seattlestyleflooring.com/wp-content/uploads/2013/12/wideskyline-1080x300.jpg")

# ANALYZING THE KING COUNTY DATASET

# Agenda

* Reading Data
* Understanding Data
* Making Assumptions
* Exploring Data
    * Location
    * Date
    * ID
    * Other Values
* Cleaning and Validating Data
* Identifying Relationsships Among Values
* Verifying Assumptions
    1. Rennovated houses will have better condition and grade
    2. Unrenovated houses sell for less than similar, but renovated houses
    3. Houses in expensive neighborhoods are more expensive per squarefoot (sqft) than similar houses in cheap neighborhoods
    4. Prices for houses will increase over time because of urbanization and inflation
    5. Prices for houses in expensive neighborhoods will increase faster than the prices in cheap neighborhoods
* Recommendations for Client
    * Limitations

# Reading Data

As a first step, we import the necessary Python packages like **pandas**, **numpy** and the general visualization packages.

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

The we read in the data from the csv-file and take a first look at the size and the first rows.

In [ ]:
df_kc = pd.read_csv("~/neuefische/Kingcounty_EDA/King_County_House_prices_dataset.csv")

print(f"The dataset contains {df_kc.shape[1]} columns and {df_kc.shape[0]:,} rows.\n")
print("It contains the following columns:", *df_kc.columns, "\n")
df_kc.head()

# Understanding Data

<details><summary>
    Click here for the column names and descriptions for Kings-County dataset
</summary>
    <ol start="0">
        <li><b>id</b> - Unique identifier for a house</li>
        <li><b>date</b> - Date teh house was sold</li>
        <li><b>price</b> - Price the house was sold for; prediction target for this notebook</li>
        <li><b>bedrooms</b> - Number of bedrooms per house</li>
        <li><b>bathrooms</b> - Number of bathrooms per house</li>
        <li><b>sqft_livingsquare</b> - Footage of the house</li>
        <li><b>sqft_lotsquare</b> - Footage of the lot</li>
        <li><b>floors</b> - Number of floors (or levels) in the house</li>
        <li><b>waterfront</b> - If 1 house has is waterfront</li>
        <li><b>view</b> - Number of times house has been viewed</li>
        <li><b>condition</b> - Overall condition of the house</li>
        <li><b>grade</b> - Overall grade given to the construction quality of the house, based on <a href="https://info.kingcounty.gov/assessor/esales/Glossary.aspx?type=r#g">King County grading system</a>, higher values are better</li>
        <li><b>sqft_above</b> - Square footage of the house apart from basement</li>
        <li><b>sqft_basement</b> - Square footage of the basement</li>
        <li><b>yr_built</b> - Year house was built</li>
        <li><b>yr_renovated</b> - Year house was last renovated</li>
        <li><b>zipcode</b> - Zip code the house is located in</li>
        <li><b>lat</b> - Latitude coordinate of the houses location</li>
        <li><b>long</b> - Longitude coordinate of the houses location</li>
        <li><b>sqft_living15</b> - Mean square footage of living space for the nearest 15 neighbors</li>
        <li><b>sqft_lot15</b> - Mean square footage of the land lots of the nearest 15 neighbors</li>
    </ol>
</details>
<br>

Now we check if the data was read in correctly. For this purpose, we are looking at the datatypes of the columns and the number of different values in the columns.

In [ ]:
df_kc.describe()

In [ ]:
df_kc.info()

From these outputs we can see that some columns have wrong datatypes or could be categories:

* **id** and **zipcode** are not for counting and should be a string
* **date** has to be a datetime-object
* **price** can be an integer
* **view** can be an integer, NaNs are assumed to mean "not viewed" an set to 0
* **waterfront** is a binary category
* **condition** and **grade** are categorical but we will leave them as integers, because they are ordered
* **sqft_basement** is broken (not displayed in describe-method) and need to be fixed
* **yr_renovated** can be an integer, NaNs are assumed to mean "not renovated" and will be put to "yr_built"

That is why we have to reset some of the datatypes.

In [ ]:
df_kc["id"] = df_kc["id"].astype("str")
df_kc["date"] = pd.to_datetime(df_kc["date"]).dt.date
df_kc["price"] = df_kc["price"].astype("int")
df_kc["waterfront"] = df_kc["waterfront"].astype("category")
df_kc["view"] = df_kc["view"].fillna(0).astype("int")
df_kc["sqft_basement"] = pd.to_numeric(df_kc["sqft_basement"], errors='coerce')
df_kc["zipcode"] = df_kc["zipcode"].astype("str")
df_kc["yr_renovated"] = df_kc["yr_renovated"].fillna(0).astype("int")

**waterfront** and **sqft_basement** will still have missing values, but we will deal with this after exploring the dataset in more detail to assume the correct values.

# Making Assumptions

The client is an investor and wants to do good. Therefore, she is looking for run-down houses in the poor neighborhoods. She wants to buy unrenovated hard-to-sell houses in these neighborhoods and then renovate them. She only wants small profits and is also willing to cross-finance. Based on this I made the following assumptions:

1. Rennovated houses will have better condition and grade.
2. Unrenovated houses sell for less than similar, but renovated houses.
3. Houses in expensive neighborhoods are more expensive per squarefoot (sqft) than similar houses in cheap neighborhoods.
4. Prices for houses will increase over time because of urbanization and inflation.
5. Prices for houses in expensive neighborhoods will increase faster than the prices in cheap neighborhoods.

# Exploring Data

## Location

Since the most important things about houses are the location, the location and the location we will take a look at the houses on the map. For this we use the folium-package. This allows us to also display some information about the houses when moving with the cursor over a house on the map.

In [1]:
# Friendly borrowed from https://medium.com/@saidakbarp/interactive-map-visualization-with-folium-in-python-2e95544d8d9b

import folium
from folium.plugins import MarkerCluster

# Set Seattle coordinates
seattle_coord = [47.5000, -122.0000]

# Build map
#map_seattle = folium.Map(location=seattle_coord, zoom_start=9, tiles='cartodbpositron', width=880, height=480)

# Add marker cluster
#marker_cluster = MarkerCluster(maxClusterRadius=60, spiderfyOnMaxZoom=False).add_to(map_seattle)

# Add house locations including tooltips
#for i in range(df_kc.shape[0]):
#    location = [df_kc["lat"][i], df_kc["long"][i]]
#    tooltip = ''.join([f'Size: {df_kc["sqft_living"][i]:,} sqft<br> Lot size: {df_kc["sqft_lot"][i]:,} sqft<br>',
#                       f'Floors: {df_kc["floors"][i]}<br>',
#                       f'Bedrooms: {df_kc["bedrooms"][i]}<br> Bathrooms: {df_kc["bathrooms"][i]}<br>',
#                       f'Sold: {df_kc["date"][i]}<br> Price: ${df_kc["price"][i]:,}<br>'
#                      ])
#    folium.Marker(location, tooltip=tooltip).add_to(marker_cluster)

# Add a title
#title_html = '''
#             <h3 align="center" style="font-size:16px">Locations of houses in King County dataset</h3>
#             '''
#map_seattle.get_root().html.add_child(folium.Element(title_html))

# Display map in Jupyter
#map_seattle

We now know that we are looking at houses in the greater Seattle area. All **lat** and **long** values seem reasonable.

## Date

Let's take a look at the **date** column to see which timespan is covered by our dataset.

In [ ]:
print(f'First sale: {df_kc["date"].min()}')
print(f'Last sale: {df_kc["date"].max()}')

Our dataset covers sales betwenn May 2, 2014 and May 27, 2015. Let's see how the sales are distributed over this year. And because we haven't done this so far we need to import matplotlib and seaborn to plot nice charts.

In [ ]:
# Count sales per date
sales_day = df_kc.groupby("date").count()["id"].reset_index()

# Filter out days weekends
sales_day = sales_day[sales_day["id"] > 20]

sns.set(rc={'figure.figsize':(15,5)})
sns.lineplot(x=sales_day["date"], y=sales_day["id"])
plt.xlabel("Date")
plt.ylabel("Number of sales per day")
plt.title("Sales per day").set_size(16)

We can see that sales are going down in January and February as well as the end of our dataset. The low numbers could be connected to high expenses during Christmas and the beginning of the new year. The step drop could be connected to the collection of data.

## ID

Let's take a look at the **id**. 

In [ ]:
print(df_kc["id"].nunique())
df_kc["id"].unique()

We see that there are only 21,420 **id**s for the houses, but there are 21,597 observations in the dataset. So some houses were sold multiple times in this timespan.

## Other Values

This is not very big data and we do not have too many features. Thus, we can plot most of them and reach some useful analytical results. This way we may detect some possible outliers or decide to do normalization. We start with the describe-method and continue with a grid plot of histograms.

In [ ]:
hist_grid = df_kc[['price', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors',
                   'waterfront', 'view', 'condition', 'grade', 'sqft_above', 'sqft_basement',
                   'yr_built', 'yr_renovated', 'sqft_living15', 'sqft_lot15'
                  ]].hist(bins=25,figsize=(15,15),xlabelsize='10',ylabelsize='10',xrot=-15);

[x.title.set_size(16) for x in hist_grid.ravel()];

# Cleaning and Validating Data

There are missing values in **waterfront**. We will fill these missing observations with 0, meaning the aren't waterfront.

In [ ]:
df_kc["waterfront"] = df_kc["waterfront"].fillna(0)

#View houses with NaN in waterfront with FastMarkerCluster

In addition, there are missing values in the column **sqft_basement**, too. We won't fill these NaNs because **sqft_basement** is a colinear variable anyways. This is because **sqft_basement** = **sqft_living** - **sqft_above**. We will just drop this variable.

In [ ]:
df_kc.drop(["sqft_basement"], axis=1, inplace=True)

Also **yr_renovated** still includes a lot of 0s. We will assume that this means these houses weren't renovated. Therefore, we are setting the renovation date to the year the house was built.

In [ ]:
for i in range(df_kc.shape[0]):
    if df_kc["yr_renovated"][i] == 0:
        df_kc["yr_renovated"][i] = df_kc["yr_built"][i]

# Identifying Relationsships Among Values

Before we look at the relationships among the values, we should introduce to new variables to get a better feeling for the prices of houses: The price per square feet of living space **pp_sqft_living** and the price per square feet of lot size **pp_sqft_lot**. These to variables will make comparing house prices easier.

In [ ]:
df_kc["pp_sqft_living"] = df_kc["price"] / df_kc["sqft_living"]

print(f'Cheapest house by sqft: {df_kc["pp_sqft_living"].min():,.2f} $/sqft')
print(f'Most expensive house by sqft: {df_kc["pp_sqft_living"].max():,.2f} $/sqft')

#sns.histplot(data=df_kc, x="pp_sqft_living");
sns.boxplot(data=df_kc, x="pp_sqft_living");

In [ ]:
df_kc["pp_sqft_lot"] = df_kc["price"] / df_kc["sqft_lot"]

print(f'Cheapest lot by sqft: {df_kc["pp_sqft_lot"].min():,.2f} $/sqft')
print(f'Most expensive lot by sqft: {df_kc["pp_sqft_lot"].max():,.2f} $/sqft')

#sns.histplot(data=df_kc, x="pp_sqft_lot");
sns.boxplot(data=df_kc, x="pp_sqft_lot");

Another important thing is correlation. If there is very high correlation between two features, keeping both of them is not a good idea most of the time not to cause overfitting. For instance, if there is overfitting, we may remove sqt_above or sqt_living because they are highly correlated. This relation can be estimated when we look at the definitions in the dataset but to be sure correlation matrix should be checked. However, this does not mean that you must remove one of the highly correlated features. For example: bathrooms and sqrt_living. They are highly correlated but I do not think that the relation among them is the same as the relation between sqt_living and sqt_above.

In [ ]:
features = ["price", "bedrooms", "bathrooms", "sqft_living", "sqft_lot", "floors",
            "waterfront", "view", "condition", "grade", "sqft_above", "yr_built",
            "yr_renovated", "zipcode", "lat", "long", "sqft_living15", "sqft_lot15",
            "pp_sqft_living", "pp_sqft_lot"
           ]

mask = np.zeros_like(df_kc[features].corr(), dtype=np.bool) 
mask[np.triu_indices_from(mask)] = True 

f, ax = plt.subplots(figsize=(15, 12))
plt.title('Pearson Correlation Matrix',fontsize=16)

sns.heatmap(df_kc[features].corr(), linewidths=0.25, cmap="Blues", annot=True, annot_kws={"size":8}, mask=mask, cbar_kws={"shrink": .9});

In [ ]:
#sns.pairplot(df_kc[["price", "bedrooms", "bathrooms", "sqft_living",
#                    "sqft_lot", "sqft_above", "sqft_living15", "sqft_lot15"
#                   ]]);

# Verifying Assumptions

## 1. Rennovated houses will have better condition and grade

As renovated houses we will count houses that were renovated or built not more than ten years before the sale. We will add an extra column to capture this.

In [ ]:
recently_renovated = []

for i in range(df_kc.shape[0]):
    if (df_kc["date"][i].year - df_kc["yr_renovated"][i]) <= 10:
        recently_renovated.append(1)
    else:
        recently_renovated.append(0)

df_kc["recently_renovated"] = recently_renovated
df_kc.recently_renovated.value_counts()

3,738 houses were renovated 10 years before they were sold. Now we group by this new variable and look at the mean **condition** and **grade**.

In [ ]:
df_kc.groupby("recently_renovated").mean()[["condition", "grade"]].reset_index()

The mean **grade** following the King County grading system is higher for recently renovated houses than for not renovated houses. This assumption is therefore correct: Renovated or fresh built houses have a higher construction quality.

## 2. Unrenovated houses sell for less than similar, but renovated houses

In [ ]:
rr_bedroom = df_kc.groupby(["bedrooms", "recently_renovated"]).mean()["pp_sqft_living"].reset_index()

sns.barplot(x="bedrooms", y="pp_sqft_living", hue="recently_renovated", data=rr_bedroom);

It looks like houses with 7 bedrooms or more have a higher value when they are renovated.

In [ ]:
rr_bathroom = df_kc.groupby(["bathrooms", "recently_renovated"]).mean()["pp_sqft_living"].reset_index()

sns.barplot(x="bathrooms", y="pp_sqft_living", hue="recently_renovated", data=rr_bathroom);

Similarly, renovated houses seem to sell for higher prices when they have less than 2.5 or more than 4.5 bathrooms. However, this is not very clear, but rather a tendency.

## 3. Houses in expensive neighborhoods are more expensive per squarefoot (sqft) than similar houses in cheap neighborhoods

Cheap neighborhoods are defined by the relatively low price of the houses. The threshold is set at $300,000.

In [ ]:
df_kc_zip = df_kc.groupby("zipcode").mean()[["price", "pp_sqft_living"]].reset_index()
df_kc_zip.sort_values(by="price", ascending=False)

In [ ]:
cheap_neighborhood = []

for i in range(df_kc_zip.shape[0]):
    if df_kc_zip["price"][i] < 300000:
        cheap_neighborhood.append(1)
    else:
        cheap_neighborhood.append(0)

df_kc_zip["cheap_neighborhood"] = cheap_neighborhood
df_kc_zip.cheap_neighborhood.value_counts()

In [ ]:
df_kc_zip.groupby("cheap_neighborhood").mean()["pp_sqft_living"].reset_index()

## 4. Prices for houses will increase over time because of urbanization and inflation

In [ ]:
# Calculate mean sales price per date
price_day = df_kc.groupby("date").mean()["price"].reset_index()

#sns.set(rc={'figure.figsize':(15,5)})
sns.lineplot(x=price_day["date"], y=price_day["price"])
plt.xlabel("Date")
plt.ylabel("Mean sales price per day")
plt.title("Mean sales price per day").set_size(16)

Sales prices have been stable over the period of the year and have not risen. However, this does not mean that prices are not generally rising. But this is probably only visible over a longer period of time.

We take a look at the houses sold multiple times: It seems that the houses are sold again for higher prices.

In [ ]:
sold_multi = df_kc.groupby("id").count()["date"].reset_index()
list_sold_multi = sold_multi[sold_multi["date"] > 1]["id"].values
df_kc[df_kc["id"].isin(list_sold_multi)].head(10)

## 5. Prices for houses in expensive neighborhoods will increase faster than the prices in cheap neighborhoods

In [ ]:
df_kc = df_kc.merge(df_kc_zip[["zipcode", "cheap_neighborhood"]], on="zipcode")
#df_kc.head()

In [ ]:
# Calculate mean sales price per date
price_day_cheap = df_kc.groupby(["date", "cheap_neighborhood"]).mean()[["price"]].reset_index()
price_day_cheap

#sns.set(rc={'figure.figsize':(15,5)})
sns.lineplot(x="date", y="price", hue="cheap_neighborhood", data=price_day_cheap)
plt.xlabel("Date")
plt.ylabel("Mean sales price per day")
plt.title("Mean sales price per day").set_size(16)

This assumption seems to be false. The price development in cheaper neighborhoods is constant and parallel to the price development in the expensive neighborhoods.

## Recommendations for Client

We are looking at the cheapest neighborhoods by total sales price:

In [ ]:
# Build new map for recommendation
#map_seattle_rec2 = folium.Map(location=seattle_coord, zoom_start=9, tiles='cartodbpositron', width=880, height=480)

# Add Choropleth map with zipcodes and mean prices per sqft
#folium.Choropleth(geo_data="wa_washington_zip_codes_geo.geojson", # source: https://github.com/OpenDataDE/State-zip-code-GeoJSON/blob/master/wa_washington_zip_codes_geo.min.json
#                  data=df_kc_zip,
#                  columns=["zipcode", "price"], # zip code is here for matching the geojson zipcode, price is the column that changes the color of zipcode areas
#                  key_on="feature.properties.ZCTA5CE10", # this path contains zipcodes in str type, this zipcodes should match with our ZIP CODE column
#                  bins=6, fill_opacity=0.6, line_opacity=0.2, nan_fill_color="PapayaWhip",
#                  legend_name="Price"
#                 ).add_to(map_seattle_rec2)

# Add a title
#title_html = '''
#             <h3 align="center" style="font-size:16px">Mean sales price by zipcode</h3>
#             '''
#map_seattle_rec2.get_root().html.add_child(folium.Element(title_html))

# Display map in Jupyter
#map_seattle_rec2

We see a lot of cheaper neighborhoods, especially in the south. It might be hard to choose the neighboorhood needing the most help. Also, total sales prices are distorted by the size of the house. So let's take a look at the prices per square footage of living space.

In [1]:
# Build new map for recommendation
#map_seattle_rec = folium.Map(location=seattle_coord, zoom_start=9, tiles='cartodbpositron', width=880, height=480)

# Add marker cluster
#marker_cluster_rec = MarkerCluster(maxClusterRadius=60, spiderfyOnMaxZoom=False).add_to(map_seattle_rec)

# Add Choropleth map with zipcodes and mean prices per sqft
#folium.Choropleth(geo_data="wa_washington_zip_codes_geo.geojson", # source: https://github.com/OpenDataDE/State-zip-code-GeoJSON/blob/master/wa_washington_zip_codes_geo.min.json
#                  data=df_kc_zip,
#                  columns=["zipcode", "pp_sqft_living"], # zip code is here for matching the geojson zipcode, price is the column that changes the color of zipcode areas
#                  key_on="feature.properties.ZCTA5CE10", # this path contains zipcodes in str type, this zipcodes should match with our ZIP CODE column
#                  bins=8, fill_opacity=0.7, line_opacity=0.2, nan_fill_color="PapayaWhip",
#                  legend_name="Price per sqft"
#                 ).add_to(map_seattle_rec)

# Add a title
#title_html = '''
#             <h3 align="center" style="font-size:16px">Mean sales price per sqft by zipcode</h3>
#             '''
#map_seattle_rec.get_root().html.add_child(folium.Element(title_html))

# Display map in Jupyter
#map_seattle_rec

Houses in the west of Seattles downtown are not as expensive in total prices as the ones in downtown. But since they seem to be smaller, their prices per square footage of living space is also very high. The same applies to the houses in the east of Seattle. Just the south has lower prices in total and per square footage.

The client should by in these areas, to really help develop cheaper neighborhoods.

Let's now take a look at the 100 cheapest houses by price per square footage:

In [ ]:
# Filter for cheapest and oldest houses in cheap_neighborhoods
df_kc_cheap = df_kc[(df_kc["cheap_neighborhood"] == 1) & (df_kc["pp_sqft_living"] < 104.3)].reset_index()

In [ ]:
# Add house locations including tooltips
for i in range(df_kc_cheap.shape[0]):
    location = [df_kc_cheap["lat"][i], df_kc_cheap["long"][i]]
    tooltip = ''.join([f'Size: {df_kc_cheap["sqft_living"][i]:,} sqft<br> Lot size: {df_kc_cheap["sqft_lot"][i]:,} sqft<br>',
                       f'Year renovated: {df_kc_cheap["yr_renovated"][i]}<br>',
                       f'Price: ${df_kc_cheap["price"][i]:,}<br> Price per sqft: ${df_kc_cheap["pp_sqft_living"][i]:,.2f}'
                      ])
    folium.Marker(location, tooltip=tooltip).add_to(marker_cluster_rec)

# Displaying map in Jupyter
map_seattle_rec

The client should buy these houses, to help develop the cheap neighborhoods.

Since there could be financial restrictions on the client, we should sort out the houses. We will sort these 100 houses by **yr_renovated** to find the most run-down houses:

In [ ]:
df_kc_cheap_old = df_kc_cheap[df_kc_cheap["cheap_neighborhood"] == 1].sort_values("yr_renovated").reset_index()
df_kc_cheap_old.drop(["level_0", "index", "date", "view", "condition", "lat", "long", "sqft_living15", "sqft_lot15", "recently_renovated", "cheap_neighborhood"], axis=1, inplace=True)
df_kc_cheap_old.head(10)

## Limitations

A statement about unsaleable houses is not possible, because the provided data set contains only data about sold houses. The profit that can be earned by renovating houses cannot be calculated because the renovation costs are unknown.